In [16]:
########################################
# PHASE 1: Lexical Analysis (Scanner)
########################################

def lexical_analysis():
    source_code = input("Enter the source code:\n")
    length = len(source_code)
    position = 0
    line = 1

    tokens = []
    st = {}  # Symbol table
    keywords = {
        "LET", "IF", "THEN", "ELSE", "ENDIF", "CALL", "WHILE",
        "FOR", "FUNC", "RETURN", "BEGIN", "END", "ENDWHILE", "DO"
    }
    logical_operators = {"AND", "OR", "NOT"}
    operators = {'+', '-', '*', '/', '<', '>', '+=', '-=', '=', '/=', '++', '--', '!=', '==', '>='}
    spaces = {' ', '\t', '\n'}
    brackets = {
        '(': 'left_paren',
        ')': 'right_paren',
        ',': 'comma',
        '{': 'left_brace',
        '}': 'right_brace',
        '[': 'left_bracket',
        ']': 'right_bracket'
    }

    open_parentheses_count = 0
    open_brackets_count = 0
    open_strings = False

    while position < length:
        current_char = source_code[position]

        # 1) Skip whitespace
        if current_char in spaces:
            if current_char == '\n':
                line += 1
            position += 1
            continue

        # 2) Handle comments delimited by { ... }
        if current_char == '{':
            position += 1
            while position < length and source_code[position] != '}':
                if source_code[position] == '\n':
                    line += 1
                position += 1
            if position == length:
                print(f"Syntax Error: Unclosed comment at line {line}, position {position}")
                break
            position += 1  # Skip '}'
            continue

        # 3) Handle string literals "..."
        if current_char == '"':
            if open_strings:
                print(f"Syntax Error: Unclosed string at line {line}, position {position}")
                break
            open_strings = True
            position += 1
            start = position
            while position < length and source_code[position] != '"':
                if source_code[position] == '\n':
                    print(f"Syntax Error: Unterminated string at line {line}, position {position}")
                    break
                position += 1
            if position == length:
                print(f"Syntax Error: Unterminated string at line {line}, position {position}")
                break
            lexeme = source_code[start:position]
            tokens.append(('string', lexeme))
            position += 1  # Skip closing '"'
            open_strings = False
            continue

        # 4) Handle identifiers / keywords / logical operators
        if current_char.isalpha() or current_char == '_':
            start = position
            while (position < length and
                   (source_code[position].isalnum() or source_code[position] == '_')):
                position += 1
            lexeme = source_code[start:position]
            upper_lexeme = lexeme.upper()
            if upper_lexeme in keywords:
                token_type = upper_lexeme.lower()
            elif upper_lexeme in logical_operators:
                token_type = 'logical_operator'
            else:
                token_type = 'identifier'
            tokens.append((token_type, lexeme))
            continue

        # 5) Handle numbers (integer or float)
        if current_char.isdigit():
            start = position
            has_decimal = False
            while position < length and (source_code[position].isdigit() or source_code[position] == '.'):
                if source_code[position] == '.':
                    if has_decimal:
                        print(f"Syntax Error: Multiple decimals in number at line {line}, position {position}")
                        break
                    has_decimal = True
                position += 1
            lexeme = source_code[start:position]
            tokens.append(('number', lexeme))
            continue

        # 6) Handle operators (+, -, *, /, <, >, !=, ==, >=, etc.)
        if current_char in '+-*/<>=!':
            start = position
            lexeme = current_char
            position += 1
            # Check for two-character operators
            if position < length and source_code[position] in '=<>':
                lexeme += source_code[position]
                position += 1
            if lexeme in operators:
                if lexeme == '=':
                    tokens.append(('equal', lexeme))
                elif lexeme == '!=':
                    tokens.append(('not_equal', lexeme))
                elif lexeme == '==':
                    tokens.append(('equal_equal', lexeme))
                elif lexeme == '>=':
                    tokens.append(('greater_equal', lexeme))
                else:
                    tokens.append(('operator', lexeme))
            else:
                tokens.append(('operator', current_char))
            continue

        # 7) Handle brackets/parentheses
        if current_char in brackets:
            token_type = brackets[current_char]
            tokens.append((token_type, current_char))
            position += 1
            if current_char == '(':
                open_parentheses_count += 1
            elif current_char == ')':
                open_parentheses_count -= 1
            elif current_char == '[':
                open_brackets_count += 1
            elif current_char == ']':
                open_brackets_count -= 1
            continue

        # Unknown character
        print(f"Syntax Error: Unknown character '{current_char}' at line {line}, position {position}")
        break

    if open_parentheses_count > 0:
        print(f"Syntax Error: Unclosed parenthesis at the end of input.")
    if open_brackets_count > 0:
        print(f"Syntax Error: Unclosed bracket at the end of input.")

    # Symbol Table Population
    i = 0
    while i < len(tokens):
        token_type, lexeme = tokens[i]

        if token_type == "let":
            i += 1
            if i < len(tokens) and tokens[i][0] == "identifier":
                var_name = tokens[i][1]
                i += 1
                if i < len(tokens) and tokens[i][0] == "equal":
                    i += 1
                    if i < len(tokens):
                        value_token = tokens[i]
                        value_type, value_lexeme = value_token

                        if value_type == 'number':
                            if '.' in value_lexeme:
                                st[var_name] = 'float'
                            else:
                                st[var_name] = 'int'
                        elif value_type == 'string':
                            st[var_name] = 'string'
                        elif value_type == 'logical_operator':
                            st[var_name] = 'bool'
                        elif value_type == 'left_bracket':
                            # Handle list assignments (e.g., LET list = [1, 2, 3])
                            list_elements = []
                            i += 1
                            while i < len(tokens) and tokens[i][0] != 'right_bracket':
                                element_type, element_lexeme = tokens[i]
                                if element_type == 'number':
                                    list_elements.append(float(element_lexeme) if '.' in element_lexeme else int(element_lexeme))
                                elif tokens[i][0] == "comma":
                                    pass
                                else:
                                    print(f"Syntax Error: Unexpected element in list at token {i}")
                                    break
                                i += 1

                            if i < len(tokens) and tokens[i][0] == "right_bracket":
                                if all(isinstance(el, int) for el in list_elements):
                                    st[var_name] = 'list of int'
                                elif all(isinstance(el, float) for el in list_elements):
                                    st[var_name] = 'list of float'
                                else:
                                    st[var_name] = 'list of mixed types'
                            else:
                                print(f"Syntax Error: Expected ']' to close list at token {i}")
                        elif value_type == 'identifier':
                            # Handle assignments from other variables or function calls
                            if i + 1 < len(tokens) and tokens[i + 1][0] == 'left_paren':
                                # Function call assignment (e.g., LET result = func(args))
                                func_name = value_lexeme
                                st[var_name] = 'unknown'  # Function return type can be determined in semantic analysis
                            else:
                                st[var_name] = st.get(value_lexeme, 'unknown')
                        elif value_type in operators:
                            print(f"Syntax Error: Incomplete expression at token {i}, position {position}")
                        else:
                            print(f"Syntax Error: Unexpected value '{value_lexeme}' at token {i}")
                    else:
                        print(f"Syntax Error: Expected value after '=' at token {i}")
                else:
                    print(f"Syntax Error: Expected '=' after identifier at token {i}")
            else:
                print(f"Syntax Error: Expected identifier after 'LET' at token {i}")

        elif token_type in ("call", "func"):
            i += 1
            if i < len(tokens) and tokens[i][0] == "identifier":
                func_name = tokens[i][1]
                params = []
                i += 1
                if i < len(tokens) and tokens[i][0] == "left_paren":
                    i += 1
                    while i < len(tokens) and tokens[i][0] != "right_paren":
                        if tokens[i][0] == "identifier":
                            param_name = tokens[i][1]
                            params.append(param_name)
                            if param_name not in st:
                                st[param_name] = 'unknown'
                        elif tokens[i][0] == "number":
                            params.append(tokens[i][1])
                        elif tokens[i][0] == "string":
                            params.append(f'"{tokens[i][1]}"')
                        elif tokens[i][0] == "comma":
                            pass
                        else:
                            print(f"Syntax Error: Expected identifier or number in function parameters at token {i}, position {position}")
                            break
                        i += 1
                    if i < len(tokens) and tokens[i][0] == "right_paren":
                        st[func_name] = f"function (with parameters: {', '.join(params)})"
                    else:
                        print(f"Syntax Error: Expected ')' after function parameters at token {i}, position {position}")
                else:
                    st[func_name] = "function (with parameters: )"
                    i -= 1
            else:
                print(f"Syntax Error: Expected function name after 'CALL' at token {i}, position {position}")
                i -= 1
        else:
            pass
        i += 1

    # Mark any identifiers not in symbol table as 'unknown'
    for token_type, lexeme in tokens:
        if token_type == 'identifier' and lexeme not in st:
            st[lexeme] = 'unknown'

    return tokens, st


########################################
# PHASE 2: Syntax Analysis (Parser)
########################################

class ParserError(Exception):
    pass

class ParseNode:
    """
    A tree node that can print itself in a vertical ASCII style.
    """
    def __init__(self, name):
        self.name = name
        self.children = []

    def add_child(self, child):
        if child is not None:
            self.children.append(child)

    def _print_tree(self, prefix="", is_last=True):
        """
        Recursively prints the tree in an ASCII structure:

        Program
        ├─ statements
        │  └─ declare_statement
        ...
        """
        branch = "└─" if is_last else "├─"
        print(prefix + branch + self.name)
        next_prefix = prefix + ("   " if is_last else "│  ")
        for i, child in enumerate(self.children):
            child_is_last = (i == len(self.children) - 1)
            child._print_tree(next_prefix, child_is_last)

    def print_tree(self):
        self._print_tree()


class RecursiveDescentParser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.current_token = self.tokens[self.pos] if self.tokens else None

    def advance(self):
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]
        else:
            self.current_token = None

    def match(self, expected_type=None):
        """Consume the current token if it matches expected_type, otherwise error."""
        if self.current_token is None:
            raise ParserError("Unexpected end of tokens while matching.")

        ttype, tlex = self.current_token
        if expected_type and ttype != expected_type:
            raise ParserError(
                f"Syntax Error: expected {expected_type}, got {ttype} (lexeme='{tlex}') at pos {self.pos}"
            )

        # Label tokens like "let: LET", "id: a", "operator: +", etc.
        label_map = {
            'let':          f"let: {tlex}",
            'identifier':   f"id: {tlex}",
            'equal':        f"equal: {tlex}",
            'operator':     f"operation: {tlex}",
            'not_equal':    f"operation: {tlex}",
            'equal_equal':  f"operation: {tlex}",
            'greater_equal': f"operation: {tlex}",
            'number':       f"number: {tlex}",
            'string':       f"string: {tlex}",
            'if':           f"if: {tlex}",
            'then':         f"then: {tlex}",
            'else':         f"else: {tlex}",
            'endif':        f"endif: {tlex}",
            'call':         f"call: {tlex}",
            'begin':        f"begin: {tlex}",
            'end':          f"end: {tlex}",
            'left_paren':   f"left_paren: {tlex}",
            'right_paren':  f"right_paren: {tlex}",
            'comma':        f"comma: {tlex}",
            'logical_operator': f"logical_operator: {tlex}"
        }
        label = label_map.get(ttype, f"{ttype}: {tlex}")

        node = ParseNode(label)
        self.advance()
        return node

    def parse(self):
        root = self.parse_program()
        if self.current_token is not None:
            ttype, tlex = self.current_token
            raise ParserError(f"Extra tokens remain: {ttype}('{tlex}') at pos {self.pos}")
        return root

    ###################################
    # Grammar Rules
    ###################################

    # Program -> begin statements end
    def parse_program(self):
        program_node = ParseNode("Program")

        # must see 'begin'
        if not self.current_token or self.current_token[0] != 'begin':
            raise ParserError("Program must start with 'BEGIN'")
        begin_node = self.match('begin')
        program_node.add_child(begin_node)

        # statements
        sb = self.parse_statements()
        program_node.add_child(sb)

        # must see 'end'
        if not self.current_token or self.current_token[0] != 'end':
            raise ParserError("Program must end with 'END'")
        end_node = self.match('end')
        program_node.add_child(end_node)

        return program_node

    # statements -> statement statements | (empty)
    def parse_statements(self):
        sb_node = ParseNode("statements")

        while self.current_token is not None:
            ttype, _ = self.current_token
            if ttype in ('end', 'endif', 'else', 'endwhile'):
                break
            stmt = self.parse_statement()
            sb_node.add_child(stmt)

        return sb_node

    # statement -> declare_statement | call_statement | if_statement | ...
    def parse_statement(self):
        if not self.current_token:
            return None

        ttype, tlex = self.current_token
        if ttype == 'let':
            return self.parse_declare_statement()
        elif ttype == 'call':
            return self.parse_call_statement()
        elif ttype == 'if':
            return self.parse_if_statement()
        else:
            raise ParserError(f"Syntax Error: unexpected token {ttype}('{tlex}') in statement")

    # declare_statement -> let identifier equal expression
    def parse_declare_statement(self):
        node = ParseNode("declare_statement")

        let_node = self.match('let')
        node.add_child(let_node)

        if not self.current_token or self.current_token[0] != 'identifier':
            raise ParserError("Expected identifier after 'let'")
        id_node = self.match('identifier')
        node.add_child(id_node)

        if not self.current_token or self.current_token[0] != 'equal':
            raise ParserError("Expected '=' in declaration")
        eq_node = self.match('equal')
        node.add_child(eq_node)

        # parse_expression might return a single child (like "number: 5")
        # or an "expression" node (if there's an operator).
        expr_node = self.parse_expression()
        node.add_child(expr_node)

        return node

    # call_statement -> call identifier ( args )
    def parse_call_statement(self):
        node = ParseNode("call_statement")

        call_node = self.match('call')
        node.add_child(call_node)

        if not self.current_token or self.current_token[0] != 'identifier':
            raise ParserError("Expected identifier after 'call'")
        func_id = self.match('identifier')
        node.add_child(func_id)

        if not self.current_token or self.current_token[0] != 'left_paren':
            raise ParserError("Expected '(' after function name")
        lp = self.match('left_paren')
        node.add_child(lp)

        if self.current_token and self.current_token[0] != 'right_paren':
            args_node = self.parse_args()
            node.add_child(args_node)

        if not self.current_token or self.current_token[0] != 'right_paren':
            raise ParserError("Expected ')' at the end of call statement")
        rp = self.match('right_paren')
        node.add_child(rp)

        return node

    # args -> expression ( comma expression )*
    def parse_args(self):
        args_node = ParseNode("args")

        first_expr = self.parse_expression()
        args_node.add_child(first_expr)

        while self.current_token and self.current_token[0] == 'comma':
            cnode = self.match('comma')
            args_node.add_child(cnode)
            e = self.parse_expression()
            args_node.add_child(e)

        return args_node

    # if_statement -> if condition then then_statements else_part? endif
    def parse_if_statement(self):
        if_node = ParseNode("if_statement")

        if_tok = self.match('if')
        if_node.add_child(if_tok)

        cond = self.parse_condition()
        if_node.add_child(cond)

        if not self.current_token or self.current_token[0] != 'then':
            raise ParserError("Expected 'THEN' after if condition")
        then_tok = self.match('then')
        then_stmt = ParseNode("then_statement")
        then_stmt.add_child(then_tok)

        # parse statements inside 'then'
        then_stmts = self.parse_statements()
        then_stmt.add_child(then_stmts)
        if_node.add_child(then_stmt)

        # optional else
        if self.current_token and self.current_token[0] == 'else':
            else_tok = self.match('else')
            else_stmt = ParseNode("else_statement")
            else_stmt.add_child(else_tok)
            else_stmts = self.parse_statements()
            else_stmt.add_child(else_stmts)
            if_node.add_child(else_stmt)

        if not self.current_token or self.current_token[0] != 'endif':
            raise ParserError("Missing 'ENDIF' at end of if statement")
        endif_tok = self.match('endif')
        if_node.add_child(endif_tok)

        return if_node

    # condition -> expression (comparison_operator expression)? (logical_operator condition)*
    def parse_condition(self):
        cond_node = ParseNode("condition")

        left_expr = self.parse_expression()
        cond_node.add_child(left_expr)

        # Comparison Operators: <, >, <=, >=, ==, !=
        if self.current_token and self.current_token[0] in ('operator', 'not_equal', 'equal_equal', 'greater_equal'):
            op_node = self.match(self.current_token[0])
            cond_node.add_child(op_node)
            right_expr = self.parse_expression()
            cond_node.add_child(right_expr)

        # Logical Operators: AND, OR, NOT
        while self.current_token and self.current_token[0] == 'logical_operator':
            log_op_node = self.match('logical_operator')
            cond_node.add_child(log_op_node)
            next_cond = self.parse_condition()
            cond_node.add_child(next_cond)

        return cond_node

    # *****************************************************
    # expression -> factor ( operator factor )*
    # If there are NO operators, we return just the factor
    # If there's at least one operator, we create an "expression" node
    # *****************************************************
    def parse_expression(self):
        left_factor = self.parse_factor()

        # Collect subsequent (operator factor) pairs
        extra_ops = []
        while self.current_token and self.current_token[0] == 'operator':
            op_lex = self.current_token[1]
            # Only parse if it's +, -, *, or /
            if op_lex not in ('+', '-', '*', '/'):
                break
            op_node = self.match('operator')
            fac_node = self.parse_factor()
            extra_ops.append(op_node)
            extra_ops.append(fac_node)

        # If NO operators, return the factor node alone (no "expression" wrap)
        if not extra_ops:
            return left_factor

        # Otherwise build an "expression" node
        expr_node = ParseNode("expression")
        expr_node.add_child(left_factor)
        for item in extra_ops:
            expr_node.add_child(item)
        return expr_node

    # factor -> number | identifier | string | ( expression )
    def parse_factor(self):
        if not self.current_token:
            raise ParserError("Unexpected end in factor.")

        ttype, tlex = self.current_token
        if ttype == 'number':
            return self.match('number')
        elif ttype == 'identifier':
            return self.match('identifier')
        elif ttype == 'string':
            return self.match('string')
        elif ttype == 'left_paren':
            lp = self.match('left_paren')
            expr = self.parse_expression()
            if not self.current_token or self.current_token[0] != 'right_paren':
                raise ParserError("Missing ')' in factor.")
            rp = self.match('right_paren')

            # Wrap in a node to keep track of parentheses
            group_node = ParseNode("expression_group")
            group_node.add_child(lp)
            group_node.add_child(expr)
            group_node.add_child(rp)
            return group_node
        else:
            raise ParserError(f"Unexpected token '{tlex}' in factor.")


########################################
# Main Driver
########################################

if __name__ == "__main__":
    tokens, st = lexical_analysis()

    print("\n=== PHASE 1 RESULTS (Scanner) ===")
    print("Tokens:")
    for token in tokens:
        token_type, lexeme = token
        print(f"  Token: {token_type}, Lexeme: {lexeme}")

    print("\nSymbol Table:")
    for name, type_ in st.items():
        print(f"  Name: {name}, Type: {type_}")

    print("\n=== PHASE 2 RESULTS (Parser) ===")
    if not tokens:
        print("No tokens to parse.")
    else:
        parser = RecursiveDescentParser(tokens)
        try:
            parse_tree = parser.parse()
            print("\nParse Tree:\n")
            parse_tree.print_tree()
        except ParserError as e:
            print("Parser Error:", e)


Enter the source code:
BEGIN     LET a = 10     LET b = 20     LET c = a + b * 2     IF c >= 50 OR c == 100 THEN         CALL process(c)     ELSE         CALL retry()     ENDIF END

=== PHASE 1 RESULTS (Scanner) ===
Tokens:
  Token: begin, Lexeme: BEGIN
  Token: let, Lexeme: LET
  Token: identifier, Lexeme: a
  Token: equal, Lexeme: =
  Token: number, Lexeme: 10
  Token: let, Lexeme: LET
  Token: identifier, Lexeme: b
  Token: equal, Lexeme: =
  Token: number, Lexeme: 20
  Token: let, Lexeme: LET
  Token: identifier, Lexeme: c
  Token: equal, Lexeme: =
  Token: identifier, Lexeme: a
  Token: operator, Lexeme: +
  Token: identifier, Lexeme: b
  Token: operator, Lexeme: *
  Token: number, Lexeme: 2
  Token: if, Lexeme: IF
  Token: identifier, Lexeme: c
  Token: greater_equal, Lexeme: >=
  Token: number, Lexeme: 50
  Token: logical_operator, Lexeme: OR
  Token: identifier, Lexeme: c
  Token: equal_equal, Lexeme: ==
  Token: number, Lexeme: 100
  Token: then, Lexeme: THEN
  Token: call, Le